In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Zaman Serisi Formatına Getirme
def create_sequences(data, target, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        targets.append(target[i + seq_length])
    return np.array(sequences), np.array(targets)

SEQ_LENGTH = 10
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_seq, y_seq = create_sequences(X_scaled, y.values, SEQ_LENGTH)

# PCA ve LDA Veri Hazırlığı
X_flat = X_seq.reshape(X_seq.shape[0], -1)  # Düzleştirilmiş veri

pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_flat)

lda = LDA(n_components=1)
X_lda = lda.fit_transform(X_flat, y_seq)

# Performans Metriklerini Hesaplama
def calculate_metrics(y_true, y_pred, y_pred_proba):
    return {
        "Accuracy": accuracy_score(y_true, y_pred) * 100,
        "Precision": precision_score(y_true, y_pred) * 100,
        "Recall": recall_score(y_true, y_pred) * 100,
        "F1-Score": f1_score(y_true, y_pred) * 100,
        "AUC": roc_auc_score(y_true, y_pred_proba) * 100
    }

# LSTM Modeli Tanımlama
def build_lstm_feature_extractor(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(64, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(32, activation='relu')  # Özellik çıkarıcı katman
    ])
    return model

# Eğitim ve Test Performansını Hesaplama
results_train = []
results_test = []

for reduction, X_data in zip(
    ['No PCA/LDA', 'PCA', 'LDA'],
    [X_seq, X_pca, X_lda]
):
    if reduction == 'No PCA/LDA':
        X_data = X_data.reshape(X_data.shape[0], SEQ_LENGTH, -1)

        # LSTM Modeli ile Özellik Çıkartma
        lstm_model = build_lstm_feature_extractor(X_data.shape[1:])
        X_features = lstm_model.predict(X_data)

    else:
        # PCA ve LDA için düzleştirilmiş veriler kullanılır
        X_features = X_data

    # Eğitim ve Test Bölünmesi
    X_train, X_test, y_train, y_test = train_test_split(X_features, y_seq, test_size=0.2, random_state=42, stratify=y_seq)

    # Random Forest Modeli
    rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
    rf_model.fit(X_train, y_train)

    # Eğitim Kümesi Performansı
    y_train_pred = rf_model.predict(X_train)
    y_train_proba = rf_model.predict_proba(X_train)[:, 1]
    train_metrics = calculate_metrics(y_train, y_train_pred, y_train_proba)
    train_metrics['Reduction'] = reduction
    train_metrics['Model'] = 'LSTM + Random Forest'
    results_train.append(train_metrics)

    # Test Kümesi Performansı
    y_test_pred = rf_model.predict(X_test)
    y_test_proba = rf_model.predict_proba(X_test)[:, 1]
    test_metrics = calculate_metrics(y_test, y_test_pred, y_test_proba)
    test_metrics['Reduction'] = reduction
    test_metrics['Model'] = 'LSTM + Random Forest'
    results_test.append(test_metrics)

# Sonuçları Görselleştirme
train_df = pd.DataFrame(results_train)
test_df = pd.DataFrame(results_test)

# Eğitim ve Test Performans Sonuçları Tablosu
print("Eğitim Kümesi Performans Sonuçları:")
print(train_df)
print("\nTest Kümesi Performans Sonuçları:")
print(test_df)

# Sonuçları Kaydetme
train_df.to_csv("lstm_rf_train_performance.csv", index=False)
test_df.to_csv("lstm_rf_test_performance.csv", index=False)
print("\nSonuçlar 'lstm_rf_train_performance.csv' ve 'lstm_rf_test_performance.csv' dosyalarına kaydedildi.")

1562/1562 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Eğitim Kümesi Performans Sonuçları:
     Accuracy   Precision      Recall    F1-Score         AUC   Reduction  \
0  100.000000  100.000000  100.000000  100.000000  100.000000  No PCA/LDA   
1  100.000000  100.000000  100.000000  100.000000  100.000000         PCA   
2   99.982485   99.982338   99.986753   99.984546   99.999893         LDA   

                  Model  
0  LSTM + Random Forest  
1  LSTM + Random Forest  
2  LSTM + Random Forest  

Test Kümesi Performans Sonuçları:
    Accuracy  Precision     Recall   F1-Score        AUC   Reduction  \
0  83.556845  84.664482  86.683151  85.661925  90.764674  No PCA/LDA   
1  82.896317  83.849974  86.471212  85.140423  89.891858         PCA   
2  80.544436  82.364206  83.557047  82.956339  89.481826         LDA   

                  Model  
0  LSTM + Random Forest  
1  LSTM + Random Forest  
2  LSTM + Random Forest  

Sonuçlar 'lstm_rf_train_performance.csv' ve 'lstm_rf_test_performance.csv' dosy